In [4]:
import numba
import numpy as np
# watch -n 0.5 nvidia-smi

In [7]:
from numba import cuda, float32

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 64

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()
        
        C[x, y] = tmp

In [8]:
N = 1000

A = np.random.random((N,N))
B = np.random.random((N,N))
C = np.zeros((N,N))
B-A

array([[-0.39177984, -0.26934988, -0.39025059, ...,  0.75243659,
         0.09340383,  0.07922009],
       [ 0.17403167, -0.44636657,  0.16764232, ...,  0.67402752,
        -0.49733722, -0.11914582],
       [ 0.12824542,  0.23822447,  0.19623596, ...,  0.26015976,
        -0.74620166,  0.23126169],
       ..., 
       [ 0.20488393, -0.52308747, -0.54503196, ..., -0.25536849,
        -0.69890661,  0.10359251],
       [ 0.12734473,  0.02689524, -0.15259972, ...,  0.50818042,
         0.76850006, -0.12653011],
       [-0.33270429,  0.34728778,  0.4827666 , ..., -0.13626936,
         0.19451288,  0.89307062]])

In [9]:
fast_matmul(A, B, C)
C

CudaAPIError: [999] Call to cuDevicePrimaryCtxRetain results in CUDA_ERROR_UNKNOWN

In [74]:
for it in range(100):
    fast_matmul(A, B, C)

In [43]:
def matmul(A, B, C):
    C = A @ B
    return C

In [44]:
%timeit matmul(A, B, C)

21.2 ms ± 3.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
fast_matmul(A, B, C)

In [34]:
4096*4

16384

In [99]:
from numba import cuda
import numpy as np

@cuda.jit
def max_example(result, values):
    """Find the maximum value in values and store in result[0]"""
    tid = cuda.threadIdx.x
    bid = cuda.blockIdx.x
    bdim = cuda.blockDim.x
    i = (bid * bdim) + tid
    cuda.atomic.max(result, 0, values[i])


arr = np.random.rand(16384+5000)
result = np.zeros(1, dtype=np.float64)

max_example[256,64](result, arr)
print(result[0]) # Found using cuda.atomic.max
print(max(arr))  # Print max(arr) for comparision (should be equal!)

0.999928298683
0.999928298683


In [100]:
%timeit max(arr)

978 µs ± 21.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [101]:
%timeit max_example[256,64](result, arr)

491 µs ± 950 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [19]:
from numba import vectorize, float64, guvectorize
import numpy as np
@vectorize([float64[:](float64[:], float64[:])], target='cuda')
def f(x, y):
    for it in range(100000):
        x[it] = y[it]
    return x

TypeError: Buffer dtype cannot be buffer

In [20]:
N = 100000+º
X = np.random.random((N))
Y = np.random.random((N))
%timeit f(X,Y)

NameError: name 'o' is not defined

In [1]:
import numpy as np
from numba import cuda, float32

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [2]:
N = 1000

A = np.random.random((N,N))
B = np.random.random((N,N))
C = np.zeros((N,N))

In [3]:
fast_matmul(A, B, C)

CudaSupportError: Error at driver init: 
[999] Call to cuInit results in CUDA_ERROR_UNKNOWN:

In [4]:
@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

In [5]:
matmul(A, B, C)

CudaSupportError: Error at driver init: 
[999] Call to cuInit results in CUDA_ERROR_UNKNOWN:

In [6]:
import torch

In [13]:
N = 10000
a = torch.rand(N,N)
b = torch.rand(N,N) 

In [ ]:
%timeit a.mm(b)

In [12]:
%timeit a@(b)

8.11 ms ± 432 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
